# Solution

## Data loading

We've already determined which pixels are "interesting" in the [previous notebook](Interesting.ipynb).

In [1]:
import pandas as pd

interesting = pd.read_pickle('data/interesting.pkl')
interesting.head()

r           c  area  eccentricity  solidity  \
part sequence frame                                                        
test 1        1       2.000000  334.000000     5      0.000000  1.000000   
              1       7.432836  339.350746   134      0.983181  0.853503   
              1      10.727273  264.642857   154      0.978166  0.865169   
              1       6.000000  321.000000     5      0.000000  1.000000   
              1      18.993590   40.365385   156      0.975910  0.901734   

                    is_satellite  
part sequence frame               
test 1        1             None  
              1             None  
              1             None  
              1             None  
              1             None

In [2]:
f'{len(interesting):,}'

'8,349,960'

## Feature extraction

In [3]:
import numpy as np
from scipy import stats

def region(img: np.ndarray, r: int, c: int, w: int):
    """Returns the square of length width with (x, y) being at the center."""
    return img[
        max(r - w, 0) : min(r + w + 1, img.shape[0]),
        max(c - w, 0) : min(c + w + 1, img.shape[1])
    ]

def extract_features(img, r, c):
    r3x3 = region(img, r, c, 3).ravel()
    r5x5 = region(img, r, c, 5).ravel()
    r7x7 = region(img, r, c, 7).ravel()
    val = img[r, c]
    return {
        'pixel_value': val,
        '3x3_std': r3x3.std(),
        '3x3_min': val - r3x3.min(),
        '3x3_max': val - r3x3.max(),
        '5x5_std': r5x5.std(),
        '5x5_entropy': stats.entropy(r5x5),
        '5x5_min': val - r5x5.min(),
        '5x5_max': val - r5x5.max(),
        '7x7_std': r7x7.std(),
        '7x7_entropy': stats.entropy(r7x7),
        '7x7_kurtosis': stats.kurtosis(r7x7),
        '7x7_skew': stats.skew(r7x7)
    }

Extract features for each interesting region.

In [33]:
from PIL import Image
import tqdm

samples = {}

# There should be 32000 frames (5 * 1280 + 5 * 5120)
for (part, sequence, frame), locations in tqdm.tqdm(interesting.groupby(['part', 'sequence', 'frame']), position=0):

#for (sequence, frame), locations in tqdm.tqdm(interesting.loc['train'].groupby(['sequence', 'frame']), position=0): 
#    part = 'train'
    
    img = np.asarray(Image.open(f'data/spotGEO/{part}/{sequence}/{frame}.png')).astype(np.float32)
    
    for _, location in locations.iterrows():
    
        r = int(location['r'])
        c = int(location['c'])

        samples[part, sequence, frame, r, c] = {
            'is_satellite': location['is_satellite'],
            'area': location['area'],
            'eccentricity': location['eccentricity'],
            'solidity':  location['solidity'],
            **extract_features(img, r=r, c=c)
        }
        
samples = pd.DataFrame.from_dict(samples, orient='index')
samples.index.names = ['part', 'sequence', 'frame', 'r', 'c']
samples.to_pickle('data/samples.pkl')
samples.head()

100%|██████████| 32000/32000 [1:33:12<00:00,  5.72it/s]  


is_satellite  area  eccentricity  solidity  \
part sequence frame r  c                                                
test 1        1     2  334         None     5      0.000000  1.000000   
                    7  339         None   134      0.983181  0.853503   
                    10 264         None   154      0.978166  0.865169   
                    6  321         None     5      0.000000  1.000000   
                    18 40          None   156      0.975910  0.901734   

                            pixel_value    3x3_std  3x3_min  3x3_max  \
part sequence frame r  c                                               
test 1        1     2  334         51.0   2.742543     12.0      0.0   
                    7  339         66.0   8.833364     26.0     -2.0   
                    10 264         94.0  18.666742     54.0     -7.0   
                    6  321         51.0   2.940476     11.0      0.0   
                    18 40          96.0  18.950119     55.0     -3.0   

                              5x5_std  5x5_entropy  5x5_min  5x5_max  \
part sequence frame r  c                                               
test 1        1     2  334   4.626940     4.472527     12.0    -16.0   
                    7  339   7.947879     4.783086     27.0     -2.0   
                    10 264  17.242287     4.750768     55.0     -7.0   
                    6  321   2.627891     4.794033     12.0      0.0   
                    18 40   17.164404     4.751998     59.0     -3.0   

                              7x7_std  7x7_entropy  7x7_kurtosis  7x7_skew  
part sequence frame r  c                                                    
test 1        1     2  334   7.047171     5.000321      1.346526  1.517550  
                    7  339   7.186401     5.405223      1.016366  1.443022  
                    10 264  15.642723     5.376112      1.676764  1.720753  
                    6  321   2.687508     5.345273      0.914122  0.866791  
                    18 40   15.657648     5.376683      1.718475  1.725620

## Learning phase

Split into train and test.

In [35]:
from sklearn import utils

X_train = samples.loc['train'].copy()
y_train = X_train.pop('is_satellite').astype(bool)
X_train, y_train = utils.shuffle(X_train, y_train, random_state=42)

try:
    X_test = samples.loc['test'].drop(columns='is_satellite')
except KeyError:
    X_test = None

Do the LGBM CV dance.

In [59]:
?lightgbm.LGBMClassifier

Init signature:
lightgbm.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    subsample_for_bin=200000,
    objective=None,
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None,
    n_jobs=-1,
    silent=True,
    importance_type='split',
    **kwargs,
)
Docstring:      LightGBM classifier.
Init docstring:
Construct a gradient boosting model.

Parameters
----------
boosting_type : string, optional (default='gbdt')
    'gbdt', traditional Gradient Boosting Decision Tree.
    'dart', Dropouts meet Multiple Additive Regression Trees.
    'goss', Gradient-based One-Side Sampling.
    'rf', Random Forest.
num_leaves : int, optional (default=31)
    Maximum tree leaves for base learners.
max_depth : int, optional (default=-1)
    Maximum tree de

In [61]:
import lightgbm
from sklearn import metrics
from sklearn import model_selection
from sklearn import utils

model = lightgbm.LGBMClassifier(
    scale_pos_weight=2,
    num_leaves=2 ** 6,
    learning_rate=.01,
    metric='binary',
    random_state=42,
    min_child_samples=30,
    n_estimators=10_000
)

cv = model_selection.GroupKFold(n_splits=5)
groups = X_train.index.get_level_values('sequence')

oof = pd.Series(dtype=bool, index=X_train.index)
if X_test is not None:
    y_test = pd.DataFrame(index=X_test.index)

for i, (fit_idx, val_idx) in enumerate(cv.split(X_train, y_train, groups=groups)):
    
    X_fit = X_train.iloc[fit_idx]
    y_fit = y_train.iloc[fit_idx]
    X_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    model.fit(
        X_fit, y_fit,
        eval_set=[(X_fit, y_fit), (X_val, y_val)],
        eval_names=['fit', 'val'],
        early_stopping_rounds=20,
        verbose=100
    )
    oof.iloc[val_idx] = model.predict(X_val)
    
    if X_test is not None:
        y_test[i] = model.predict_proba(X_test)[:, 1]
    
    print()

print(metrics.classification_report(y_train, oof, digits=4))

Training until validation scores don't improve for 20 rounds
[100]	fit's binary_logloss: 0.010353	val's binary_logloss: 0.0111353
[200]	fit's binary_logloss: 0.00797899	val's binary_logloss: 0.00920705
[300]	fit's binary_logloss: 0.00679675	val's binary_logloss: 0.00850492
[400]	fit's binary_logloss: 0.00611734	val's binary_logloss: 0.00817978
[500]	fit's binary_logloss: 0.00559571	val's binary_logloss: 0.00797129
[600]	fit's binary_logloss: 0.00522142	val's binary_logloss: 0.00783527
[700]	fit's binary_logloss: 0.0049254	val's binary_logloss: 0.00774381
[800]	fit's binary_logloss: 0.00468476	val's binary_logloss: 0.00768711
[900]	fit's binary_logloss: 0.00446799	val's binary_logloss: 0.00764279
[1000]	fit's binary_logloss: 0.00426088	val's binary_logloss: 0.00759326
[1100]	fit's binary_logloss: 0.00406992	val's binary_logloss: 0.00755357
[1200]	fit's binary_logloss: 0.0038895	val's binary_logloss: 0.00751615
[1300]	fit's binary_logloss: 0.00372246	val's binary_logloss: 0.00748116
[140

precision    recall  f1-score   support

       False     0.9987    0.9996    0.9991   1683412
        True     0.9084    0.7537    0.8239      8899

    accuracy                         0.9983   1692311
   macro avg     0.9536    0.8766    0.9115   1692311
weighted avg     0.9982    0.9983    0.9982   1692311

Feature importances.

In [62]:
pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

7x7_skew        16226
3x3_std         15515
7x7_kurtosis    14378
pixel_value     13211
7x7_entropy     12990
7x7_std         12600
5x5_entropy     11923
5x5_std         10648
eccentricity     8375
area             8329
3x3_min          7552
5x5_min          6739
solidity         5105
3x3_max          4495
5x5_max          3051
dtype: int32

## Out-of-fold predictions

In [63]:
%run toolbox.py

In [64]:
oof.head()

sequence  frame  r    c  
17        5      152  623    False
506       4      253  440    False
866       1      162  226    False
238       3      88   432    False
516       1      401  92     False
dtype: bool

In [65]:
save_predictions(oof[oof].sort_index(), path='oof.json', n_sequences=1280)

100%|██████████| 1280/1280 [00:02<00:00, 510.05it/s]


In [66]:
!python validation.py oof.json data/spotGEO/train_anno.json

Score: 0.276092, (MSE: 44956.294525)


In [ ]:
Score: 0.276092, (MSE: 44956.294525)

## Test predictions

In [67]:
y_test.head()

0         1         2         3         4
sequence frame r  c                                                    
1        1     2  334  0.000298  0.000424  0.000567  0.000758  0.000713
               7  339  0.000080  0.000072  0.000063  0.000129  0.000108
               10 264  0.000244  0.000069  0.000192  0.000126  0.000172
               6  321  0.000535  0.000689  0.000592  0.000541  0.001030
               18 40   0.000227  0.000070  0.000190  0.000145  0.000179

In [70]:
import zipfile

sightings = y_test.mean(axis='columns') > .5
save_predictions(sightings[sightings], path='submission.json', n_sequences=5120)

with zipfile.ZipFile('submission.zip', mode='w') as f:
    f.write('submission.json')

100%|██████████| 5120/5120 [00:10<00:00, 475.08it/s]


Next is [PostProcessing.ipynb](PostProcessing.ipynb).